In [1]:
from github import Github, RateLimitExceededException
import pandas as pd
import os
from datetime import datetime

from repository import Repository

In [2]:
# Read Token
access_token = open('GITHUB_TOKEN.txt', 'r').read()
user = Github(access_token)
user.per_page = 100

In [3]:
df = pd.read_json ('./data/java_best.json')

In [4]:
def check_rate_limit():
    rate_limit = user.get_rate_limit()
    remaining = rate_limit.search.remaining
    
    if remaining > 0:
        return
    
    reset_time = rate_limit.search.reset
    now = datetime.utcnow()

    if now < reset_time:
        sleep_time = (reset_time - now).total_seconds()
        print('=> Waiting {} seconds for rate limit reset.'.format(sleep_time))
        time.sleep(sleep_time)

In [5]:
for i in range (800, 801):
    print("================ {} ================".format(i))
    try:
        r = user.get_repo(int(df['id'][i]))
    except RateLimitExceededException:
        check_rate_limit()
        continue
        
    repo = Repository(r, './repos/', './analysisBestj.json')
    repo.download()
    repo.analyse()
    repo.dump()
    repo.delete()

================ 800 ================
Cloned Repository:  appformer
Analysed Repository:  appformer
Dumped data from Repository: appformer (into file: ./analysisBestj.json)
Deleted Repository:  appformer
